In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv("E:\\leonhardt\\mayi\\data\\train.csv")
df_test = pd.read_csv("E:\\leonhardt\\mayi\\data\\test_a.csv")

In [9]:
from sklearn.metrics import roc_auc_score,roc_curve
def atec_metric(preds, labels):
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1]

In [3]:
df_train=df_train.loc[df_train['label']!=-1]
df_train = df_train.fillna(0)
train_set = df_train[df_train.loc[:, "date"]<=20171005]
eva_set = df_train[df_train.loc[:, "date"]>20171005]
train_label = train_set.loc[:, "label"]
eva_label = eva_set.loc[:, "label"]
train_set.drop(["id","date", "label"], axis=1, inplace=True)
eva_set.drop(["id","date", "label"], axis=1, inplace=True)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
df_test = df_test.fillna(0)
df_test.drop(["date","id"],axis=1, inplace=True)

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
# df_eval = pd.Series(index=range(10, 380, 30))
for num in range(100, 1000, 200):
    gbdt = GradientBoostingClassifier(  
        init=None,  
        learning_rate=0.05,  
        loss='deviance',
        max_depth=8,  
        max_features=None,  
        max_leaf_nodes=None,  
        min_samples_leaf=1,  
        min_samples_split=2,  
        min_weight_fraction_leaf=0.0,  
        n_estimators=num,  
        random_state=None,  
        subsample=1.0,  
        verbose=0,  
        warm_start=False,
    )  
    gbdt.fit(train_set, train_label)
    eva_prob = gbdt.predict_proba(eva_set)
    eva_score = atec_metric(eva_prob[:, 1], eva_label)
    print("{0} score is {1}".format(num, eva_score))
    y_preds = gbdt.predict_proba(df_test)[:, 1]
    np.savetxt(os.getcwd()+"/prediction/gbdt_half0"+str(num)+".txt", y_preds, delimiter=',')

100 score is ('atec_metric', 0.15097418026294945)
300 score is ('atec_metric', 0.25228892760969424)
500 score is ('atec_metric', 0.013210834785363536)
700 score is ('atec_metric', 0.019150958339933466)
900 score is ('atec_metric', 0.24829716458102327)


In [40]:
y_preds = gbdt.predict_proba(df_test)

In [41]:
np.savetxt(os.getcwd()+"/prediction/gbdt_half_mean_160.txt", y_preds, delimiter=',')